In [1]:
include("../lib/Lib.jl")
include("../lib/DeepSpace.jl")

Main.DeepSpace

In [2]:
using DataFrames, DataFramesMeta
using CSV
using Statistics

In [3]:
KEY = "2019chs_wk1"
YEAR = "2019"
COLUMN = :teleopSwitchOwnershipSec

:teleopSwitchOwnershipSec

In [4]:
(headers, scores, teams) = Lib.read_stacked(KEY, YEAR)
scores[:color] = repeat(["r", "b"], size(scores,1) >> 1)
teams[:color] = repeat(["r", "b"], size(teams,1) >> 1)

360-element Array{String,1}:
 "r"
 "b"
 "r"
 "b"
 "r"
 "b"
 "r"
 "b"
 "r"
 "b"
 "r"
 "b"
 "r"
 ⋮  
 "r"
 "b"
 "r"
 "b"
 "r"
 "b"
 "r"
 "b"
 "r"
 "b"
 "r"
 "b"

In [5]:
DeepSpace.calculated_columns!(scores)
scores[:ownPoints] = scores.totalPoints .- scores.foulPoints .- (3 .* scores.foulCount) .- (9 * scores.techFoulCount)

360-element Array{Int64,1}:
  62
  82
  72
  71
  66
  65
  52
  54
  62
  35
  59
 -25
  63
   ⋮
  20
  12
  70
  45
  59
  47
  57
  60
  74
  66
  51
  55

In [30]:
scores_train = @linq scores |> where(:level .== "qm")
teams_train = @linq teams |> where(:level .== "qm")

scores_red = @linq scores_train |> where(:color .== "r")
scores_blue = @linq scores_train |> where(:color .== "b")
teams_red = @linq teams_train |> where(:color .== "r") |> select(:team1, :team2, :team3, :key)
teams_blue = @linq teams_train |> where(:color .== "b") |> select(:team1, :team2, :team3, :key)

,team1,team2,team3,key
,String⍰,String⍰,String⍰,String⍰
1,frc620,frc339,frc5338,2019vahay_qm1_blue
2,frc2028,frc316,frc2363,2019vahay_qm10_blue
3,frc3274,frc339,frc2534,2019vahay_qm11_blue
4,frc888,frc2186,frc5549,2019vahay_qm12_blue
5,frc611,frc612,frc6584,2019vahay_qm13_blue
6,frc1123,frc620,frc4505,2019vahay_qm14_blue
7,frc614,frc339,frc2421,2019vahay_qm15_blue
8,frc1629,frc6504,frc6584,2019vahay_qm16_blue
9,frc4464,frc3941,frc3274,2019vahay_qm17_blue


In [31]:
scoreline(teams, scores) = let stacked = stack(teams, [:team1, :team2, :team3], variable_name=:role, value_name=:team)
    join(scores, stacked, on=:key)
end

scoreline_blue = scoreline(teams_blue, scores_blue)
scoreline_red = scoreline(teams_red, scores_red)

,adjustPoints,autoPoints,bay1,bay2,bay3,bay4,bay5,bay6,bay7,bay8,cargoPoints,completeRocketRankingPoint,completedRocketFar,completedRocketNear,endgameRobot1,endgameRobot2,endgameRobot3,foulCount,foulPoints,habClimbPoints,habDockingRankingPoint,habLineRobot1,habLineRobot2,habLineRobot3,hatchPanelPoints,lowLeftRocketFar,lowLeftRocketNear,lowRightRocketFar,lowRightRocketNear,midLeftRocketFar,midLeftRocketNear,midRightRocketFar,midRightRocketNear,preMatchBay1,preMatchBay2,preMatchBay3,preMatchBay6,preMatchBay7,preMatchBay8,preMatchLevelRobot1,preMatchLevelRobot2,preMatchLevelRobot3,rp,sandStormBonusPoints,techFoulCount,teleopPoints,topLeftRocketFar,topLeftRocketNear,topRightRocketFar,topRightRocketNear,totalPoints,key,level,event,color,rocketLowPoints,rocketMidPoints,rocketTopPoints,rocketPoints,ownPoints,role,team
,Int64⍰,Int64⍰,String⍰,String⍰,String⍰,String⍰,String⍰,String⍰,String⍰,String⍰,Int64⍰,Bool⍰,Bool⍰,Bool⍰,String⍰,String⍰,String⍰,Int64⍰,Int64⍰,Int64⍰,Bool⍰,String⍰,String⍰,String⍰,Int64⍰,String⍰,String⍰,String⍰,String⍰,String⍰,String⍰,String⍰,String⍰,String⍰,String⍰,String⍰,String⍰,String⍰,String⍰,String⍰,String⍰,String⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,String⍰,String⍰,String⍰,String⍰,Int64⍰,String⍰,String⍰,String⍰,String,Int64,Int64,Int64,Int64,Int64,Symbol,String⍰
1,0,9,Panel,Panel,PanelAndCargo,None,None,PanelAndCargo,PanelAndCargo,PanelAndCargo,12,false,false,false,HabLevel1,HabLevel1,HabLevel1,0,0,9,false,CrossedHabLineInSandstorm,CrossedHabLineInTeleop,CrossedHabLineInSandstorm,2,None,None,Panel,None,None,None,None,None,Panel,Panel,Panel,Panel,Panel,Panel,HabLevel2,HabLevel2,HabLevel1,2,9,0,23,None,None,None,None,32,2019vahay_qm1_red,qm,2019vahay,r,2,0,0,2,32,team1,frc4505
2,0,9,Panel,Panel,PanelAndCargo,None,None,PanelAndCargo,PanelAndCargo,PanelAndCargo,12,false,false,false,HabLevel1,HabLevel1,HabLevel1,0,0,9,false,CrossedHabLineInSandstorm,CrossedHabLineInTeleop,CrossedHabLineInSandstorm,2,None,None,Panel,None,None,None,None,None,Panel,Panel,Panel,Panel,Panel,Panel,HabLevel2,HabLevel2,HabLevel1,2,9,0,23,None,None,None,None,32,2019vahay_qm1_red,qm,2019vahay,r,2,0,0,2,32,team2,frc2988
3,0,9,Panel,Panel,PanelAndCargo,None,None,PanelAndCargo,PanelAndCargo,PanelAndCargo,12,false,false,false,HabLevel1,HabLevel1,HabLevel1,0,0,9,false,CrossedHabLineInSandstorm,CrossedHabLineInTeleop,CrossedHabLineInSandstorm,2,None,None,Panel,None,None,None,None,None,Panel,Panel,Panel,Panel,Panel,Panel,HabLevel2,HabLevel2,HabLevel1,2,9,0,23,None,None,None,None,32,2019vahay_qm1_red,qm,2019vahay,r,2,0,0,2,32,team3,frc3941
4,0,6,None,Panel,Panel,None,PanelAndCargo,PanelAndCargo,PanelAndCargo,None,9,false,false,false,None,HabLevel1,HabLevel1,0,0,6,false,None,CrossedHabLineInSandstorm,CrossedHabLineInSandstorm,2,None,None,None,None,None,None,None,None,Cargo,Panel,Panel,Panel,Panel,Cargo,None,HabLevel1,HabLevel1,0,6,0,17,None,None,None,None,23,2019vahay_qm10_red,qm,2019vahay,r,0,0,0,0,23,team1,frc2421
5,0,6,None,Panel,Panel,None,PanelAndCargo,PanelAndCargo,PanelAndCargo,None,9,false,false,false,None,HabLevel1,HabLevel1,0,0,6,false,None,CrossedHabLineInSandstorm,CrossedHabLineInSandstorm,2,None,None,None,None,None,None,None,None,Cargo,Panel,Panel,Panel,Panel,Cargo,None,HabLevel1,HabLevel1,0,6,0,17,None,None,None,None,23,2019vahay_qm10_red,qm,2019vahay,r,0,0,0,0,23,team2,frc3941
6,0,6,None,Panel,Panel,None,PanelAndCargo,PanelAndCargo,PanelAndCargo,None,9,false,false,false,None,HabLevel1,HabLevel1,0,0,6,false,None,CrossedHabLineInSandstorm,CrossedHabLineInSandstorm,2,None,None,None,None,None,None,None,None,Cargo,Panel,Panel,Panel,Panel,Cargo,None,HabLevel1,HabLevel1,0,6,0,17,None,None,None,None,23,2019vahay_qm10_red,qm,2019vahay,r,0,0,0,0,23,team3,frc1719
7,0,3,None,None,None,None,None,None,None,None,9,false,false,false,HabLevel1,HabLevel1,None,1,0,6,false,CrossedHabLineInTeleop,CrossedHabLineInSandstorm,None,8,None,None,None,None,None,PanelAndCargo,None,Panel,Cargo,Cargo,Cargo,Cargo,Cargo,Cargo,HabLevel1,HabLevel1,None,0,3,0,23,None,PanelAndCargo,No

In [32]:
numericcols = Lib.numeric_col_names(scoreline_red)
diff_red = convert(Matrix,scoreline_red[:,numericcols]) .- convert(Matrix, scoreline_blue[:,numericcols])
diff_blue = convert(Matrix,scoreline_blue[:,numericcols]) .- convert(Matrix, scoreline_red[:,numericcols])

diffdf = DataFrame(vcat(diff_red, diff_blue), numericcols)
diffdf[:key] = vcat(scoreline_red.key, scoreline_blue.key)
diffdf[:team] = vcat(scoreline_red.team, scoreline_blue.team)

864-element Array{Union{Missing, String},1}:
 "frc4505"
 "frc2988"
 "frc3941"
 "frc2421"
 "frc3941"
 "frc1719"
 "frc623" 
 "frc1418"
 "frc4464"
 "frc2961"
 "frc5338"
 "frc4242"
 "frc6543"
 ⋮        
 "frc1599"
 "frc3258"
 "frc2998"
 "frc1598"
 "frc346" 
 "frc1086"
 "frc1522"
 "frc2998"
 "frc422" 
 "frc1080"
 "frc1598"
 "frc539" 

In [33]:
win(pointsMargin) = 
    if pointsMargin > 0 
        1.0
    elseif pointsMargin < 0 
        0.0
    else 
        0.5
    end

diffdf[:win] = win.(diffdf.totalPoints)
push!(numericcols, :win)

22-element Array{Symbol,1}:
 :adjustPoints              
 :autoPoints                
 :cargoPoints               
 :completeRocketRankingPoint
 :completedRocketFar        
 :completedRocketNear       
 :foulCount                 
 :foulPoints                
 :habClimbPoints            
 :habDockingRankingPoint    
 :hatchPanelPoints          
 :rp                        
 :sandStormBonusPoints      
 :techFoulCount             
 :teleopPoints              
 :totalPoints               
 :rocketLowPoints           
 :rocketMidPoints           
 :rocketTopPoints           
 :rocketPoints              
 :ownPoints                 
 :win                       

In [34]:
plusminus = by(diffdf, :team, Dict(n => mean for n in numericcols)...)

,team,adjustPoints_mean,foulPoints_mean,cargoPoints_mean,completeRocketRankingPoint_mean,rocketPoints_mean,rp_mean,sandStormBonusPoints_mean,rocketLowPoints_mean,rocketMidPoints_mean,teleopPoints_mean,win_mean,autoPoints_mean,completedRocketNear_mean,habDockingRankingPoint_mean,completedRocketFar_mean,totalPoints_mean,hatchPanelPoints_mean,foulCount_mean,habClimbPoints_mean,ownPoints_mean,rocketTopPoints_mean,techFoulCount_mean
,String⍰,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,frc4505,0.0,-3.45455,0.0,0.0,0.818182,-0.363636,2.72727,-1.09091,1.54545,0.0,0.454545,2.72727,0.0,-0.181818,0.0,-0.727273,1.09091,0.545455,-1.09091,-0.545455,0.363636,0.181818
2,frc2988,0.0,-0.818182,-0.545455,0.0,0.727273,0.636364,-0.272727,2.0,-0.909091,3.18182,0.636364,-0.272727,0.0,0.0909091,0.0,2.09091,1.27273,0.272727,2.45455,2.09091,-0.363636,0.0
3,frc3941,0.0,-2.81818,1.09091,0.0,-3.27273,-1.18182,-1.90909,-3.18182,0.272727,-8.45455,0.272727,-1.90909,0.0,-0.272727,0.0,-13.1818,-3.81818,0.636364,-5.72727,-13.0909,-0.363636,0.0909091
4,frc2421,0.0,-0.363636,-2.72727,0.0,0.454545,-1.81818,-3.81818,0.636364,-0.363636,-8.72727,0.136364,-3.81818,0.0,-0.363636,0.0,-12.9091,-1.63636,-0.181818,-4.36364,-12.8182,0.181818,0.0909091
5,frc1719,0.0,2.81818,-0.545455,0.0909091,1.90909,0.454545,-0.272727,0.181818,1.09091,5.81818,0.545455,-0.272727,0.0,0.181818,0.0,8.36364,2.54545,-0.636364,3.81818,8.27273,0.636364,-0.0909091
6,frc623,0.0,-0.272727,-1.09091,0.0,1.0,-0.181818,0.545455,0.0909091,0.363636,-1.81818,0.5,0.545455,0.0,-0.181818,0.0,-1.54545,1.45455,0.0909091,-2.18182,-1.54545,0.545455,0.0
7,frc1418,0.0,-0.909091,1.63636,0.0909091,15.1818,0.727273,-1.09091,5.54545,4.36364,11.2727,0.636364,-1.09091,0.0,0.0909091,0.0,9.27273,8.54545,0.0,1.09091,9.36364,5.27273,0.0909091
8,frc4464,0.0,-1.36364,-4.90909,0.0,-4.18182,-1.18182,-4.09091,-3.27273,-1.18182,-16.0909,0.272727,-4.09091,0.0,-0.272727,0.0,-21.5455,-3.81818,0.454545,-7.36364,-21.5455,0.272727,0.0
9,frc2961,0.0,0.363636,-3.54545,0.0909091,-3.54545,0.0,-1.09091,-3.54545,0.0,-5.72727,0.5,-1.09091,0.0,-0.0909091,0.0,-6.45455,-1.09091,0.181818,-1.09091,-6.54545,0.0,-0.0909091


In [35]:
showbest(col) = sort(plusminus, col, rev=true)[:,[:team, col]]
showworst(col) = sort(plusminus, col)[:,[:team, col]]

showbest(:win_mean)

,team,win_mean
,String⍰,Float64
1,frc612,1.0
2,frc2534,0.909091
3,frc614,0.909091
4,frc346,0.875
5,frc1610,0.833333
6,frc977,0.833333
7,frc339,0.818182
8,frc1262,0.75
9,frc4099,0.727273


In [36]:
using CSV
CSV.write("../output/$(KEY)_plusminus.csv", plusminus)

"../output/2019chs_wk1_plusminus.csv"